# Initialize Variables

In [1]:
import numpy as np
import math
import random
import pandas as pd

# Assigning number of bits for each component
num_courses = 15
num_sections = 10
num_professors = 10
num_days = 5
num_timeslots_per_day = 6
num_rooms = 30
max_room_size = 200
max_section_strength = 120  # Assuming a maximum section strength

# Calculate number of bits for each component using logarithm function
course_bits = math.ceil(math.log2(num_courses))
section_bits = math.ceil(math.log2(num_sections))
section_strength_bits = math.ceil(math.log2(max_section_strength))
professor_bits = math.ceil(math.log2(num_professors))
day_bits = math.ceil(math.log2(num_days))  # 5 days in a week
timeslots_bits = math.ceil(math.log2(num_timeslots_per_day))
room_bits = math.ceil(math.log2(num_rooms))

# Generate list of sections in unique binary representation
sections_binary = [format(i, 'b').zfill(section_bits) for i in range(num_sections)]
sections_strength_binary = [format(i, 'b').zfill(section_strength_bits) for i in range(max_section_strength)]
courses_binary = [format(i, 'b').zfill(course_bits) for i in range(num_courses)]
professors_binary = [format(i, 'b').zfill(professor_bits) for i in range(num_professors)]
days_binary = [format(i, 'b').zfill(day_bits) for i in range(num_days)]
rooms_binary = [format(i, 'b').zfill(room_bits) for i in range(num_rooms)]
timeslots_binary = [format(i, 'b').zfill(timeslots_bits) for i in range(num_timeslots_per_day)]
theory_lab_binary = ['0', '1']
room_size_binary = ['0', '1']  # 0 represents room_size 60, 1 represents room_size 120

# Create a list of tuples (course_binary, theory/lab_binary)
global_courses = [(course, np.random.choice(theory_lab_binary)) for course in courses_binary]

# Create professor_courses list
professor_courses = []

# Convert global_courses to a list of strings
global_courses_str = [course[0] for course in global_courses]

# Assign courses to professors
for professor in professors_binary:
    # Randomly select up to three courses for each professor
    num_assigned_courses = np.random.randint(1, 3)
    assigned_courses = np.random.choice(global_courses_str, size=num_assigned_courses, replace=False)
    for course in assigned_courses:
        # Find the corresponding tuple from global_courses using the course string
        course_tuple = next(item for item in global_courses if item[0] == course)
        professor_courses.append((course_tuple[0], course_tuple[1], professor))

# Create section_courses list
section_courses = []

# Assign courses to sections
for section in sections_binary:
    # Randomly select the number of courses for each section, up to a maximum of 5
    num_assigned_courses = np.random.randint(1, 6)
    # Shuffle the list of professor_courses to assign courses randomly
    random.shuffle(professor_courses)
    assigned_courses = professor_courses[:num_assigned_courses]
    for course, theory_lab, professor in assigned_courses:
        # Append tuples to section_courses
        section_courses.append((course, theory_lab, section, 
                                format(np.random.randint(1, max_section_strength + 1), 'b').zfill(section_strength_bits), 
                                professor))

# Create global_rooms list
global_rooms = [(room, np.random.choice(room_size_binary)) for room in rooms_binary]

# Create a list of tuples (days, timeslots)
days_and_timeslots = [(day, timeslot) for day in days_binary for timeslot in timeslots_binary]

# Print the list of section courses
# print("Section Courses:")
# for course in section_courses:
#     print(course)

# # Print the list of professor courses
# print("\nProfessor Courses:")
# for course in professor_courses:
#     print(course)
        
# print("\nRooms:")
# for room in global_rooms:
#     print(room)

# print("\nSlots:")
# for slot in days_and_timeslots:
#     print(slot)
    

# Fitness Function

In [16]:
def calculate_fitness(population):
    fitness_scores = []
    for chromosome in population:
        fitness_score = 0
        
        # Comparing first and second lecture
        if (chromosome[5] == chromosome[9] and   # Day
            chromosome[6] == chromosome[10] and  # Timeslot
            chromosome[7] == chromosome[11] and  # Room
            chromosome[1] != '1'):    # Lab Check
            fitness_score -= 1
        
        # Convert strength size binary to integer
        section_strength = int(chromosome[3], 2)
        # Convert room size binary to integer
        room_size = int(chromosome[8], 2)

        # Check if strength size is less than 60 and room size is 1
        if section_strength < 60 and room_size == 1:
            fitness_score -= 1
        # Check if strength size is more than or equal to 60 and room size is 0
        elif section_strength >= 60 and room_size == 0:
            fitness_score -= 1

        # Convert strength size binary to integer for second lecture
        section_strength2 = int(chromosome[3], 2)
        # Convert room size binary to integer for second lecture
        room_size2 = int(chromosome[12], 2)

        # Check if strength size is less than 60 and room size is 1 for second lecture
        if section_strength2 < 60 and room_size2 == 1:
            fitness_score -= 1
        # Check if strength size is more than or equal to 60 and room size is 0 for second lecture
        elif section_strength2 >= 60 and room_size2 == 0:
            fitness_score -= 1
        
        for other_chromosome in population:
            if chromosome != other_chromosome:
                
                # ----- Comparing timeslots for first lecture -----
                if (chromosome[5] == other_chromosome[5] and  # Day of first lecture
                    chromosome[6] == other_chromosome[6] and  # Timeslot of first lecture
                    chromosome[7] == other_chromosome[7]):    # Room
                    fitness_score -= 1
                    
                if (chromosome[5] == other_chromosome[9] and  # Day of second lecture
                    chromosome[6] == other_chromosome[10] and # Timeslot of second lecture
                    chromosome[7] == other_chromosome[11]):   # Room
                    fitness_score -= 1
                   
                # ----- Comparing timeslots for second lecture -----
                if (chromosome[9] == other_chromosome[5] and  # Day of first lecture
                    chromosome[10] == other_chromosome[6] and  # Timeslot of first lecture
                    chromosome[11] == other_chromosome[7]):    # Room
                    fitness_score -= 1
                    
                if (chromosome[9] == other_chromosome[9] and  # Day of second lecture
                    chromosome[10] == other_chromosome[10] and # Timeslot of second lecture
                    chromosome[11] == other_chromosome[11]):   # Room
                    fitness_score -= 1
                    
                # ----- Comparing Professor (Professor cannot teach 2 courses at one time) -----
                if (chromosome[4] == other_chromosome[4] and  # Professor of first lecture
                    chromosome[5] == other_chromosome[5] and  # Day of first lecture
                    chromosome[6] == other_chromosome[6]):    # Timeslot of first lecture
                    fitness_score -= 1
                
                if (chromosome[4] == other_chromosome[4] and   # Professor of second lecture
                    chromosome[9] == other_chromosome[9] and   # Day of second lecture
                    chromosome[6] == other_chromosome[10]):    # Timeslot of second lecture
                    fitness_score -= 1
                    
                # ----- Comparing Adjacent Days (Courses cannot be allotted to adjacent days) -----
                # Checks for first day lecture
                current_day_index = days_binary.index(chromosome[5])
                prev_day_index = (current_day_index - 1) % num_days
                next_day_index = (current_day_index + 1) % num_days
                
                prev_day_binary = days_binary[prev_day_index]
                next_day_binary = days_binary[next_day_index]
                

                if (prev_day_binary == other_chromosome[5] and  # Previous day of first lecture
                    chromosome[4] == other_chromosome[4] and     # Professor
                    chromosome[2] == other_chromosome[2] and     # Section
                    chromosome[0] == other_chromosome[0] and     # Course
                    chromosome[1] == other_chromosome[1]):       # Theory/Lab
                    fitness_score -= 1
                    
                if (next_day_binary == other_chromosome[5] and  # Next day of first lecture
                    chromosome[4] == other_chromosome[4] and     # Professor
                    chromosome[2] == other_chromosome[2] and     # Section
                    chromosome[0] == other_chromosome[0] and     # Course
                    chromosome[1] == other_chromosome[1]):       # Theory/Lab
                    fitness_score -= 1
                    
                if (prev_day_binary == other_chromosome[9] and  # Previous day of second lecture
                    chromosome[4] == other_chromosome[4] and     # Professor
                    chromosome[2] == other_chromosome[2] and     # Section
                    chromosome[0] == other_chromosome[0] and     # Course
                    chromosome[1] == other_chromosome[1]):       # Theory/Lab
                    fitness_score -= 1
                    
                if (next_day_binary == other_chromosome[9] and  # Next day of second lecture
                    chromosome[4] == other_chromosome[4] and     # Professor
                    chromosome[2] == other_chromosome[2] and     # Section
                    chromosome[0] == other_chromosome[0] and     # Course
                    chromosome[1] == other_chromosome[1]):       # Theory/Lab
                    fitness_score -= 1
                    
                # Checks for second day lecture
                current_day_index = days_binary.index(chromosome[9])
                prev_day_index = (current_day_index - 1) % num_days
                next_day_index = (current_day_index + 1) % num_days
                
                prev_day_binary = days_binary[prev_day_index]
                next_day_binary = days_binary[next_day_index]
                

                if (prev_day_binary == other_chromosome[5] and  # Previous day of first lecture
                    chromosome[4] == other_chromosome[4] and     # Professor
                    chromosome[2] == other_chromosome[2] and     # Section
                    chromosome[0] == other_chromosome[0] and     # Course
                    chromosome[1] == other_chromosome[1]):       # Theory/Lab
                    fitness_score -= 1
                    
                if (next_day_binary == other_chromosome[5] and  # Next day of first lecture
                    chromosome[4] == other_chromosome[4] and     # Professor
                    chromosome[2] == other_chromosome[2] and     # Section
                    chromosome[0] == other_chromosome[0] and     # Course
                    chromosome[1] == other_chromosome[1]):       # Theory/Lab
                    fitness_score -= 1
                    
                if (prev_day_binary == other_chromosome[9] and  # Previous day of second lecture
                    chromosome[4] == other_chromosome[4] and     # Professor
                    chromosome[2] == other_chromosome[2] and     # Section
                    chromosome[0] == other_chromosome[0] and     # Course
                    chromosome[1] == other_chromosome[1]):       # Theory/Lab
                    fitness_score -= 1
                    
                if (next_day_binary == other_chromosome[9] and  # Next day of second lecture
                    chromosome[4] == other_chromosome[4] and     # Professor
                    chromosome[2] == other_chromosome[2] and     # Section
                    chromosome[0] == other_chromosome[0] and     # Course
                    chromosome[1] == other_chromosome[1]):       # Theory/Lab
                    fitness_score -= 1
                    

        fitness_scores.append(fitness_score)
    return fitness_scores

# Initialize Population

In [17]:
# Initialize population size
pop_size = 15

# Generate chromosomes
population = []
for _ in range(pop_size):
    chromosome = []
    # Randomly select a course, its theory/lab binary, section, section strength, and professor from section_courses
    course, theory_lab, section, section_strength, professor = random.choice(section_courses)
    # Randomly select a room and its size from global_rooms
    room, room_size = random.choice(global_rooms)
    room2, room_size2 = random.choice(global_rooms)
    
    # Randomly select days and timeslots for lectures
    lecture1_day, lecture1_timeslot = random.choice(days_and_timeslots)
    
    # Check if theory_lab is equal to 1
    if theory_lab == '1':
        # Find the index of the current (day, timeslot) tuple in the days_and_timeslots list
        current_index = days_and_timeslots.index((lecture1_day, lecture1_timeslot))
        
        # Increment the index and handle wrapping around if necessary
        next_index = (current_index + 1) % len(days_and_timeslots)
        
        # Retrieve the new (day, timeslot) tuple for lecture 2
        lecture2_day, lecture2_timeslot = days_and_timeslots[next_index]
        
        room2, room_size2 = room, room_size
        
    else:
        # If theory_lab is not equal to 1, select a random (day, timeslot) for lecture 2
        lecture2_day, lecture2_timeslot = random.choice(days_and_timeslots)
    
    # Append components to chromosome
    # Course
    chromosome.append(course)
    # Theory/Lab
    chromosome.append(theory_lab)
    # Section
    chromosome.append(section)
    # Section Strength
    chromosome.append(section_strength)
    # Professor
    chromosome.append(professor)
    # First lecture day and timeslot
    chromosome.append(lecture1_day)
    chromosome.append(lecture1_timeslot)
    # First lecture room
    chromosome.append(room)
    # First lecture room size
    chromosome.append(room_size)
    # Second lecture day and timeslot
    chromosome.append(lecture2_day)
    chromosome.append(lecture2_timeslot)
    # Second lecture room
    chromosome.append(room2)
    # Second lecture room size
    chromosome.append(room_size2)
    
    population.append(chromosome)


    
# ----------- Display Population -----------
# Calculate fitness scores for the population
display_fitness_scores = calculate_fitness(population)
display_population = pd.DataFrame(population, columns=["Course", "Theory/Lab", "Section", "Section Strength",
                                                      "Professor", "Day 1", "Timeslot 1", "Room 1", "Room Size 1",
                                                      "Day 2", "Timeslot 2", "Room 2", "Room Size 2"]) 
display_fitness = pd.DataFrame({"Fitness Score": display_fitness_scores})

# Concatenate DataFrames
display_data = pd.concat([display_population, display_fitness], axis=1)

display_data

,Course,Theory/Lab,Section,Section Strength,Professor,Day 1,Timeslot 1,Room 1,Room Size 1,Day 2,Timeslot 2,Room 2,Room Size 2,Fitness Score
0,1011,0,1000,1011101,0100,010,101,11101,0,011,100,10101,1,-2
1,0101,1,0101,0110010,0001,010,000,01001,0,010,001,01001,0,0
2,0000,1,0101,1100001,0110,011,101,10000,0,100,000,10000,0,-2
3,1011,0,0000,0011001,0100,010,001,10110,0,011,010,01010,1,-2
4,1100,1,0000,0010111,0101,000,001,00110,1,000,010,00110,1,-2
5,0110,1,0001,0000101,0011,010,010,00000,1,010,011,00000,1,-2
6,1011,0,0000,0011001,0100,010,000,01101,0,000,001,00001,1,-2
7,1100,1,0000,0010111,0101,011,100,00000,1,011,101,00000,1,-2
8,0101,1,0011,0010100,1000,001,100,00101,1,001,101,00101,1,-2
9,1010,0,0011,1100000,1001,011,001,01111,1,011,100,01010,1,0


In [18]:
for _ in range(3):
    # Calculate fitness scores for the population
    fitness_scores = calculate_fitness(population)

    # Convert chromosomes and fitness scores into DataFrame
    df_population = pd.DataFrame(population, columns=["Course", "Theory/Lab", "Section", "Section Strength",
                                                      "Professor", "Day 1", "Timeslot 1", "Room 1", "Room Size 1",
                                                      "Day 2", "Timeslot 2", "Room 2", "Room Size 2"])
    df_fitness = pd.DataFrame({"Fitness Score": fitness_scores})

    # Concatenate DataFrames
    df_combined = pd.concat([df_population, df_fitness], axis=1)

    # Select 5 random rows from the DataFrame for Tournament Selection
    random_rows = df_combined.sample(n=6)

    # Sort the table by Fitness Score in descending order
    df_combined_sorted = df_combined.sort_values(by="Fitness Score", ascending=False)

    # ------------------ Selecting Successors ------------------
    # Delete the last 6 rows from df_combined_sorted
    df_combined_sorted = df_combined_sorted.iloc[:-6]
    #----------------------------------------------------------- 

    # ------------------ Selecting Parents for Crossover ------------------
    # Sort the randomly selected rows by the "Fitness Score" column in descending order
    sorted_random_rows = random_rows.sort_values(by="Fitness Score", ascending=False)
    #---------------------------------------------------------------------- 

    # ------------------ Crossover ------------------
    # Define the columns to switch
    columns_to_switch = ['Day 1', 'Timeslot 1', 'Room 1', 'Room Size 1', 'Day 2', 'Timeslot 2', 'Room 2', 'Room Size 2']

    # Iterate over rows in pairs
    for i in range(0, len(sorted_random_rows), 2):
        # Select two rows at a time
        row1 = sorted_random_rows.iloc[i].copy()  # Make a copy to avoid modifying the original DataFrame
        row2 = sorted_random_rows.iloc[i + 1].copy()  # Make a copy to avoid modifying the original DataFrame

        # Switch the values for specified columns
        for column in columns_to_switch:
            temp = row1[column]
            row1[column] = row2[column]
            row2[column] = temp

        # Update the original DataFrame with the modified rows
        sorted_random_rows.iloc[i] = row1
        sorted_random_rows.iloc[i + 1] = row2
    #-----------------------------------------------    

    # ---- Mutation ----
    # Randomly select a row index from sorted_random_rows
    random_row_index = random.randint(0, len(sorted_random_rows) - 1)

    if (sorted_random_rows.iloc[random_row_index, 1] != '1'): 
        # ------------ Mutation for Theory ------------
        # Randomly choose between replacing the 5th and 6th columns or the 9th and 10th columns
        replace_choice = random.choice(['first', 'second', 'third'])

        # Randomly select an index from days_and_timeslots
        random_timeslot_index = random.randint(0, len(days_and_timeslots) - 1)
        random_room_index = random.randint(0, len(global_rooms) - 1)


        # Get the tuple from days_and_timeslots at the randomly selected index
        random_timeslot = days_and_timeslots[random_timeslot_index]
        random_room = global_rooms[random_room_index]

        # Replace the corresponding columns in the randomly selected row based on the choice
        if replace_choice == 'first':
            # Replace 1st lecture slots
            sorted_random_rows.iloc[random_row_index, 5] = random_timeslot[0]  # Day
            sorted_random_rows.iloc[random_row_index, 6] = random_timeslot[1]  # Timeslot
        elif replace_choice == 'second':
            # Replace 2nd lecture slots
            sorted_random_rows.iloc[random_row_index, 9] = random_timeslot[0]  # Day
            sorted_random_rows.iloc[random_row_index, 10] = random_timeslot[1]  # Timeslot
        elif replace_choice == 'third':
            sorted_random_rows.iloc[random_row_index, 7] = random_room[0]  # Room
            sorted_random_rows.iloc[random_row_index, 8] = random_room[1]  # Room Size
        else:
            sorted_random_rows.iloc[random_row_index, 11] = random_room[0]  # Room
            sorted_random_rows.iloc[random_row_index, 12] = random_room[1]  # Room Size
    else:
    # ------------ Mutation for Lab ------------
        # Randomly choose between replacing the 5th and 6th columns or the 9th and 10th columns
        replace_choice = random.choice(['first', 'second'])

        # Randomly select an index from days_and_timeslots
        random_timeslot_index = random.randint(0, len(days_and_timeslots) - 1)
        random_room_index = random.randint(0, len(global_rooms) - 1)


        # Get the tuple from days_and_timeslots at the randomly selected index
        random_timeslot = days_and_timeslots[random_timeslot_index]
        random_timeslot2 = days_and_timeslots[random_timeslot_index+1]
        random_room = global_rooms[random_room_index]

        # Replace the corresponding columns in the randomly selected row based on the choice
        if replace_choice == 'first':
            # Replace 1st lecture slots
            sorted_random_rows.iloc[random_row_index, 5] = random_timeslot[0]  # Day
            sorted_random_rows.iloc[random_row_index, 6] = random_timeslot[1]  # Timeslot
            # Replace 2nd lecture slots
            sorted_random_rows.iloc[random_row_index, 9] = random_timeslot2[0]  # Day
            sorted_random_rows.iloc[random_row_index, 10] = random_timeslot2[1]  # Timeslot
        elif replace_choice == 'second':
            sorted_random_rows.iloc[random_row_index, 7] = random_room[0]  # Room 1
            sorted_random_rows.iloc[random_row_index, 8] = random_room[1]  # Room 1 Size
            sorted_random_rows.iloc[random_row_index, 11] = random_room[0]  # Room 2
            sorted_random_rows.iloc[random_row_index, 12] = random_room[1]  # Room 2 Size
    # ------------------------------------------------------
    

    # Drop the 'Fitness Score' column
    sorted_random_rows = sorted_random_rows.drop(columns=['Fitness Score'])
    df_combined_sorted = df_combined_sorted.drop(columns=['Fitness Score'])

    # # Convert the DataFrame back to a list
    modified_rows_list = sorted_random_rows.values.tolist()
    population = df_combined_sorted.values.tolist()
    population.extend(modified_rows_list)

 
# Convert binary string values back to integers
df_combined_int = df_combined.map(lambda x: int(x, 2) if isinstance(x, str) and set(x) <= {'0', '1'} else x)

# Display the DataFrame with integer values
df_combined_int

,Course,Theory/Lab,Section,Section Strength,Professor,Day 1,Timeslot 1,Room 1,Room Size 1,Day 2,Timeslot 2,Room 2,Room Size 2,Fitness Score
0,5,1,5,50,1,2,0,9,0,2,1,9,0,0
1,14,1,6,64,2,4,4,3,1,4,5,3,1,0
2,12,1,7,12,5,3,2,25,0,3,3,25,0,0
3,6,1,1,5,3,0,1,11,0,0,2,11,0,0
4,11,0,8,93,4,2,5,29,0,3,4,21,1,-5
5,0,1,5,97,6,3,5,16,0,4,0,16,0,-2
6,12,1,0,23,5,0,1,6,1,0,2,6,1,-2
7,12,1,0,23,5,2,2,0,1,2,3,0,1,-2
8,10,0,3,96,9,3,1,15,1,3,4,10,1,-4
9,11,0,8,93,4,3,1,15,1,3,4,10,1,-4


In [14]:
df_combined

,Course,Theory/Lab,Section,Section Strength,Professor,Day 1,Timeslot 1,Room 1,Room Size 1,Day 2,Timeslot 2,Room 2,Room Size 2,Fitness Score
0,0100,1,0000,0101111,0000,100,011,01011,0,100,100,01011,0,-2
1,1011,1,0001,1101111,0111,010,010,11001,1,010,011,11001,1,-2
2,0101,1,0001,0100101,0111,010,011,01101,0,010,100,01101,0,-1
3,1001,1,0011,1000111,0011,100,010,01100,0,100,011,01100,0,-4
4,1010,1,0010,1101110,0101,001,010,10111,0,001,011,10111,0,-4
5,0000,0,0111,0111110,0000,010,010,01000,0,000,100,00010,0,-3
6,0101,1,0110,0110010,0111,100,000,01010,1,100,001,01010,1,-2
7,0000,0,0001,0110101,0000,000,001,00010,0,010,101,10011,0,-1
8,0100,1,0001,1101110,0000,011,101,00011,1,100,000,00011,1,0
9,0100,1,0000,0101111,0000,010,010,11001,1,010,011,11001,1,-6
